# Lab 4

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fayam1/Geospatial-Data-Management-course/blob/main/lab_04.ipynb)

## Submission requirements

1. Upload a screenshot of your map for each question.
2. Provide a link to your notebook on Colab. See instructions [here](https://geog-414.gishub.org/book/labs/instructions.html).

## Datasets

The datasets being used in the lab are listed below:

- [TIGER: US Census Counties](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_Counties)
- [Landsat-9](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2)
- [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
- [NAIP](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ)

## Question 1

Write a program to find out how many counties are named `Knox` in the US.

## There are 9 counties named Knox

In [59]:
# Add your code here
import ee
import geemap
import geopandas as gpd

# Initialize the Earth Engine API
ee.Initialize()

# Load the US counties 
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Filter counties by name "Knox"
knox_counties = counties.filter(ee.Filter.eq('NAME', 'Knox'))

# Convert  to a GeoPandas GeoDataFrame
gdf=geemap.ee_to_gdf(knox_counties)

# Print the GeoDataFrame
gdf


,geometry,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP
0,"POLYGON ((-90.44436 40.71489, -90.44400 40.714...",1855569287,8837382,23660,H1,095,00424249,,A,17095,+40.9309446,-090.2137921,06,,G4020,Knox,Knox County,17
1,GEOMETRYCOLLECTION (LINESTRING (-87.24997 38.5...,1336511196,20758105,47180,H1,083,00450366,,A,18083,+38.6883910,-087.4203544,06,,G4020,Knox,Knox County,18
2,GEOMETRYCOLLECTION (LINESTRING (-83.87762 36.6...,1000523138,3743713,30940,H1,121,00516907,,A,21121,+36.8874755,-083.8556297,06,,G4020,Knox,Knox County,21
3,"POLYGON ((-69.45123 44.31394, -69.45114 44.313...",945684684,2017116243,,H1,013,00581292,,A,23013,+44.0420446,-069.0385145,06,,G4020,Knox,Knox County,23
4,"POLYGON ((-92.34925 40.23975, -92.34876 40.237...",1305376626,7307094,,H1,103,00758506,,A,29103,+40.1368559,-092.1468345,06,,G4020,Knox,Knox County,29
5,"POLYGON ((-98.31038 42.81264, -98.31034 42.812...",2870856403,81009708,,H1,107,00835875,,A,31107,+42.6344045,-097.8913492,06,,G4020,Knox,Knox County,31
6,"POLYGON ((-82.75074 40.27701, -82.74710 40.276...",1361052752,10720134,34540,H1,083,01074054,198,A,39083,+40.4036198,-082.4223929,06,,G4020,Knox,Knox County,39
7,"POLYGON ((-84.27349 35.93926, -84.27340 35.938...",1316572232,45558452,28940,H1,093,01639761,314,A,47093,+35.9927265,-083.9377209,06,,G4020,Knox,Knox County,47
8,"POLYGON ((-99.99642 33.83423, -99.99629 33.829...",2203109026,12577260,,H1,275,01383923,,A,48275,+33.6118897,-099.7303563,06,,G4020,Knox,Knox County,48


In [3]:
knox_counties.size()

![](https://i.imgur.com/3Jg9P6X.png)

## Question 2

Display Knox county of Tennesse with outline only (no fill color) on the map. (Hint: The `STATEFP` of Tennessee is `47`)

In [60]:
# Add your code here
import ee
import geemap
import geopandas as gpd

# Initialize the Earth Engine API
ee.Initialize()

# Load the US counties 
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Filter counties by name "Knox" and STATEFP=47
knox_Tennesse = counties.filter(ee.Filter.And
                                (ee.Filter.eq('NAME','Knox'),
                                 ee.Filter.eq('STATEFP','47')
                                ))
# Define the styling for the outline (no fill)
style = {
    'color': 'red',  # Outline color
    'fillColor': '00000000',  # Transparent fill color
    'width': 2  # Outline width
}

map=geemap.Map()
map.addLayer(knox_Tennesse.style(**style),{},'knox Tennesse') 
# Add text to the map
text = "Made by Faissal Bozi"
map.add_text(text, fontsize=20, position='bottomright')
map.centerObject(knox_Tennesse)
map



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/28Iaw9b.png)

## Question 3

Use [Landsat-9](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC09_C02_T1_L2) data to create a cloud-free imagery for Knox County, TN. Display the imagery on the map with a proper band combination.

In [61]:
# Add your code here
collection=ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")\
             .filterBounds(knox_Tennesse)\
             .filterMetadata("CLOUD_COVER","less_than",5)
             

# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )           
image1=collection.map(apply_scale_factors).median().clip(knox_Tennesse)
# set the vis parameters
vis = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # True color (RGB) combination
    'min': 0.0,
    'max': 0.30,
    'gamma':1.8
   
}
map1=geemap.Map()
# Add text to the map
text = "Made by Faissal Bozi"
map1.add_text(text, fontsize=20, position='bottomright')
#Add layers
map1.addLayer(image1,vis,'Landsat 9')
map1.addLayer(knox_Tennesse.style(**style),{},'knox Tennesse')
map1.centerObject(knox_Tennesse)
map1

         




Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/m72amN0.png)

## Question 4

Use [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) data to create a cloud-free imagery for Knox County, TN. Display the imagery on the map with a proper band combination.

In [62]:
# Add your code here
#define and filter the image collection
collection1=ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
              .filterBounds(knox_Tennesse)\
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5))
#Clip the image to Knox county boundary
s2_image=collection1.median().clip(knox_Tennesse)
map2=geemap.Map()
#define the vis
vis={'min':0.0,
     'max':3000,
     'bands':['B8','B3','B2']}
#Add  layers
map2.addLayer(s2_image,vis,'Sentenel-2 Imagery')
map2.addLayer(knox_Tennesse.style(**style),{},'knox Tennesse')
#Add text
text = "Made by Faissal Bozi"
map2.add_text(text, fontsize=20, position='bottomright')
map2.centerObject(knox_Tennesse)
#display the map
map2



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/vM1M8Gc.png)

## Question 5

Use [NAIP](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ) imagery to create a cloud-free imagery for Knox County, TN. Display the imagery on the map with a proper band combination.

In [83]:
# Add your code here
collection2=ee.ImageCollection("USDA/NAIP/DOQQ")\
               .filterBounds(knox_Tennesse)\
               .filterDate('2020-01-01', '2024-03-30')
#Define vis 
nimage=collection2.median().clip(knox_Tennesse)
vis={'bands': ['N', 'R', 'G'],
    'min': 0,
    'max': 255,
    'gamma': 1.4,
}

m=geemap.Map()
#Add  layers
m.addLayer(knox_Tennesse.style(**style),{},'knox Tennesse')
m.addLayer(nimage,vis, 'Naip image')
#Add text
text = "Made by Faissal Bozi"
m.add_text(text, fontsize=20, position='bottomright')
m.centerObject(knox_Tennesse)
m


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/iZSGqGS.png)